# Optimizing for CO2 storage from CO2 Emitters

## Setup

In [1]:
import numpy as np
import pandas as pd
from pulp import *
from geopy.distance import distance
from geopy.distance import great_circle
import random
import pickle

In [29]:
def facility_cleaning(fac_df, emiss_df):
    facilitiesB = fac_df.rename(inplace=False,
                                    columns={'Facility Name':'facility_name',
                                             'City':'city',
                                             'Primary NAICS Code':'naics_code',
                                             'Industry Type (subparts)':'industry_subparts',
                                             'Industry Type (sectors)':'industry_sectors'})

    emsCA2019loc = pd.merge(emiss_df, facilitiesB[['Facility Id',
                                                    'facility_name',
                                                   'Latitude',
                                                   'Longitude',
                                                   'supply_metric',
                                                   'industry_subparts',
                                                   'industry_sectors']],
                            how='left', on='Facility Id')
    emsCA2019locagg = emsCA2019loc[['Facility Id','facility_name','Latitude','Longitude',
                                    'supply_metric']].dropna().groupby(
                                        ['Facility Id','facility_name']).agg({'Latitude':'min',
                                                            'Longitude':'min',
                                                            'supply_metric':'sum'})
    emsCA2019locagg.reset_index(inplace=True)
    emsCA2019locagg = emsCA2019locagg.rename(columns={'Facility Id':'facility_id',
                                                      'facility_name':'facility_name',
                                                      'Latitude':'lat',
                                                      'Longitude':'lon'})
    emsCA2019locagg = emsCA2019locagg[emsCA2019locagg['supply_metric'] > 0]
    facility_latlon = emsCA2019locagg[['lat', 'lon']].to_records(index=False)
    facility_ids = emsCA2019locagg['facility_id'].tolist()
    facility_names = emsCA2019locagg['facility_name'].tolist()
    supply_volume = emsCA2019locagg['supply_metric'].tolist()
    supply_trucks = [i/25 for i in supply_volume]
    return facility_latlon, facility_ids, facility_names, supply_volume, supply_trucks


def get_distances(facility_latlon, pools_latlon):
    pools_distance_matrix = []
    for i in range(len(facility_latlon)):
        distances = []
        for j in range(len(pools_latlon)):
            dist = great_circle(facility_latlon[i], pools_latlon[j]).miles
            distances.append(dist)
        pools_distance_matrix.append(distances)
    return pools_distance_matrix


def get_complex_costs(distance_matrix, pool_trucks):
    complex_costs = []
    for i in range(len(distance_matrix)):
        new = []
        for j in range(len(distance_matrix[i])):
            opening_cost = 2000000/pool_trucks[j]
            # opening cost = $2,000,000/number of trucks for the pool
            # cost / truck = 2*$2*distance + opening cost
            cost = 4 * distance_matrix[i][j] + opening_cost
            new.append(cost)
        complex_costs.append(new)
    return complex_costs


def run_optimization(n_receivers, n_suppliers, supply_trucks, demand_trucks, costs, distances, facility_names, facility_latlon, pools_ids, pools_latlon, industry_str, year):
    print("Running optimization for {} industry and {} year timeframe".format(industry_str, year))
    supply_trucks = [i*year for i in supply_trucks]
    
    p_prob = LpProblem('Unbalaced_Transportation_Problem', LpMinimize)
    #costs = pools_distance_matrix_complexcost
    routes = [(i, j) for i in range(n_suppliers) for j in range(n_receivers)]

    x = LpVariable.dicts('X', routes, lowBound=0)
    p_prob += lpSum([x[i, j] * costs[i][j] for i in range(n_suppliers) for j in range(n_receivers)])
    
    if sum(supply_trucks) > sum(demand_trucks):
        for i in range(n_suppliers):
            p_prob += lpSum([x[i, j] for j in range(n_receivers)]) <= supply_trucks[i]

        for j in range(n_receivers):
            p_prob += lpSum([x[i, j] for i in range(n_suppliers)]) == demand_trucks[j]
    else:
        for i in range(n_suppliers):
            p_prob += lpSum([x[i, j] for j in range(n_receivers)]) == supply_trucks[i]

        for j in range(n_receivers):
            p_prob += lpSum([x[i, j] for i in range(n_suppliers)]) <= demand_trucks[j]

    # Solving problem
    p_prob.solve()
    print("Problem successfully solved")
    
    volumes_moved = [i.varValue for i in p_prob.variables()]
    volume_matrix = []
    k=0
    while k < n_suppliers:
        vols = []
        for i in range(n_receivers):
            j = i + k*n_receivers
            vols.append(volumes_moved[j])
        volume_matrix.append(vols)
        k+=1
    print("Volume matrix created")
        
    results = []
    for i in range(n_suppliers):
        f_name = facility_names[i]
        f_loc = facility_latlon[i]
        for j in range(n_receivers):
            vol = volume_matrix[i][j]
            if vol > 0:
                w_id = pools_ids[j]
                w_loc = pools_latlon[j]
                dist = distances[i][j]
                cost = costs[i][j] * vol
                r = [f_name, f_loc, w_id, w_loc, dist, cost, vol, industry_str, year]
                results.append(r)
    results_df = pd.DataFrame(results, columns = ['facility_name', 'facility_location','pool_id','pool_location','distance','cost','n_trucks', 'industry', 'timeframe'])
    print("Results created")
    print()
    print("--------------------------------------------")
    
    return results_df


## Data import and cleaning

In [3]:
wells = pd.read_csv("data/wells/AllWells_20210915.csv")

# Oil & Gas, Dry Gas (no liquids), Gas and "Liquefied Gas"
wells_oilgas = wells[ (wells['WellType'] == 'OG')  |
                      (wells['WellType'] == 'DG')  |
                      (wells['WellType'] == 'GAS') |
                      (wells['WellType'] == 'LG') ]
wells_latlon = wells_oilgas[['Latitude', 'Longitude']].to_records(index=False)
wells_ids = wells_oilgas['API'].tolist()
pooled_wells = pd.read_csv("data/pool_volumes.csv")
pools_latlon = pooled_wells[['Latitude', 'Longitude']].to_records(index=False)
pools_ids = pooled_wells['pool_id'].tolist()
pools_volumes = pooled_wells['totalco2'].tolist()
pools_trucks = [i/25 for i in pools_volumes]
n_pools = len(pools_ids)

In [4]:
emissions = pd.read_csv(r'./data/Emissions by Unit and Fuel Type - csv/UNIT_DATA-Table 1.csv')
emsCA2019 = emissions[emissions['Reporting Year'] == 2019]
facilities = pd.read_excel(r'./data/ghgp_data_2019.xlsx',
                          sheet_name='Direct Emitters', skiprows=3)
# One Michegan emitter lists its CA headquarters. Retag it as Michigan
facilities.loc[facilities['Facility Id'] == 1000594, 'State'] = 'MI'

## Prepare data for optimization

In [5]:
facilities = facilities[(facilities.State == 'CA')]
facilities_cement = facilities[facilities['Cement Production'] > 0]
facilities_cement['supply_metric'] = facilities_cement['Cement Production']
facilities_oil = facilities[facilities['Petroleum Refining'] > 0]
facilities_oil['supply_metric'] = facilities_oil['Petroleum Refining']
facilities_energy = facilities[facilities['Electricity Generation'] > 0]
facilities_energy['supply_metric'] = facilities_energy['Electricity Generation']
facilities_steel = facilities[facilities['Iron and Steel Production'] > 0]
facilities_steel['supply_metric'] = facilities_steel['Iron and Steel Production']

facility_c_latlon, facility_c_ids, facility_c_names, supply_c_volume, supply_c_trucks = facility_cleaning(facilities_cement, emsCA2019)
n_facilities_c = len(facility_c_ids)

facility_o_latlon, facility_o_ids, facility_o_names, supply_o_volume, supply_o_trucks = facility_cleaning(facilities_oil, emsCA2019)
n_facilities_o = len(facility_o_ids)

facility_e_latlon, facility_e_ids, facility_e_names, supply_e_volume, supply_e_trucks = facility_cleaning(facilities_energy, emsCA2019)
n_facilities_e = len(facility_e_ids)

facility_s_latlon, facility_s_ids, facility_s_names, supply_s_volume, supply_s_trucks = facility_cleaning(facilities_steel, emsCA2019)
n_facilities_s = len(facility_s_ids)

facility_total_latlon = np.concatenate((facility_c_latlon, facility_o_latlon, facility_e_latlon, facility_s_latlon))
facility_total_ids = np.concatenate((facility_c_ids, facility_o_ids, facility_e_ids, facility_s_ids))
facility_total_names = np.concatenate((facility_c_names, facility_o_names, facility_e_names, facility_s_names))
supply_volume_total = np.concatenate((supply_c_volume, supply_o_volume, supply_e_volume, supply_s_volume))
supply_trucks_total = np.concatenate((supply_c_trucks, supply_o_trucks, supply_e_trucks, supply_s_trucks))
n_facilities_total = len(facility_total_ids)

<ipython-input-5-ad7731123256>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facilities_cement['supply_metric'] = facilities_cement['Cement Production']
<ipython-input-5-ad7731123256>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facilities_oil['supply_metric'] = facilities_oil['Petroleum Refining']
<ipython-input-5-ad7731123256>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [6]:
facilitiesB = facilities_cement.rename(inplace=False,
                                    columns={'Facility Name':'facility_name',
                                             'City':'city',
                                             'Primary NAICS Code':'naics_code',
                                             'Industry Type (subparts)':'industry_subparts',
                                             'Industry Type (sectors)':'industry_sectors'})
emsCA2019loc = pd.merge(emsCA2019, facilitiesB[['Facility Id',
                                                    'facility_name',
    #                                               'Facility Name2',
                                                   'city',
                                                   'Zip Code',
                                                   'Address',
                                                   'County',
                                                   'Latitude',
                                                   'Longitude',
    #                                               'Primary NAICS Code2',
                                                   'industry_subparts',
                                                   'industry_sectors']],
                            how='left', on='Facility Id')
emsCA2019locagg = emsCA2019loc[['Facility Id','facility_name','Latitude','Longitude',
                                    'Unit CO2 emissions (non-biogenic)']].dropna().groupby(
                                        ['Facility Id','facility_name']).agg({'Latitude':'min',
                                                            'Longitude':'min',
                                                            'Unit CO2 emissions (non-biogenic)':'sum'})
emsCA2019locagg.reset_index(inplace=True)
emsCA2019locagg = emsCA2019locagg.rename(columns={'Facility Id':'facility_id',
                                                      'facility_name':'facility_name',
                                                      'Latitude':'lat',
                                                      'Longitude':'lon',
                                                      'Unit CO2 emissions (non-biogenic)':'annual_emissions'})
emsCA2019locagg = emsCA2019locagg[emsCA2019locagg['annual_emissions'] > 0]
emsCA2019locagg

,facility_id,facility_name,lat,lon,annual_emissions
0,1002308,CEMEX Construction Materials Pacific LLC,34.622200,-117.100100,12834.3
1,1002431,HANSON PERMANENTE CEMENT,37.318100,-122.091000,464.9
2,1004612,LEHIGH SOUTHWEST CEMENT CO.,40.736900,-122.322300,1072.2
3,1005662,Mitsubishi Cement Corp Cushenbury Cement Plant,34.437557,-116.891034,421.7
4,1006642,NATIONAL CEMENT CO OF CALIFORNIA INC,34.819863,-118.748732,2476.8
5,1006842,CalPortland Company Mojave Plant,35.029298,-118.316236,712.0
6,1007927,CalPortland Company Oro Grande Plant,34.604500,-117.338200,270.5


In [7]:
distances_c = get_distances(facility_c_latlon, pools_latlon)
complex_costs_c = get_complex_costs(distances_c, pools_trucks)

distances_o = get_distances(facility_o_latlon, pools_latlon)
complex_costs_o = get_complex_costs(distances_o, pools_trucks)

distances_e = get_distances(facility_e_latlon, pools_latlon)
complex_costs_e = get_complex_costs(distances_e, pools_trucks)

distances_s = get_distances(facility_s_latlon, pools_latlon)
complex_costs_s = get_complex_costs(distances_s, pools_trucks)

distances_total = get_distances(facility_total_latlon, pools_latlon)
complex_costs_total = get_complex_costs(distances_total, pools_trucks)

In [8]:
industry_list = ['cement', 'oil_refining', 'steel', 'energy', 'total']
year_list = [1, 5, 10, 20, 30]

inputs_list = []
for i in year_list:
    cement = [n_facilities_c, supply_c_trucks, complex_costs_c, distances_c, facility_c_names, facility_c_latlon, 'cement', i]
    oil = [n_facilities_o, supply_o_trucks, complex_costs_o, distances_o, facility_o_names, facility_o_latlon, 'oil_refining', i]
    energy = [n_facilities_e, supply_e_trucks, complex_costs_e, distances_e, facility_e_names, facility_e_latlon, 'energy', i]
    steel = [n_facilities_s, supply_s_trucks, complex_costs_s, distances_s, facility_s_names, facility_s_latlon, 'steel', i]
    total = [n_facilities_total, supply_trucks_total, complex_costs_total, distances_total, facility_total_names, facility_total_latlon, 'total', i]
    
    inputs_list.append(cement)
    inputs_list.append(oil)
    inputs_list.append(steel)
    inputs_list.append(energy)
    inputs_list.append(total)

    

## Run optimization and write results

In [30]:
detailed_results = pd.DataFrame()

for i in inputs_list:
    results = run_optimization(n_pools,
                               i[0],
                               i[1],
                               pools_trucks,
                               i[2],
                               i[3],
                               i[4],
                               i[5],
                               pools_ids,
                               pools_latlon,
                               i[6],
                               i[7])
    detailed_results = detailed_results.append(results)


Running optimization for cement industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for oil_refining industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for steel industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for energy industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for total industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for cement industry and 5 year timeframe
Problem successfully solved
Volume matrix created
Results create

In [31]:
# re calculate cost as 2 * ($2*miles)*(volume/25) + 2,000,000
detailed_results['volume'] = detailed_results['n_trucks']*25
detailed_results['cost_per_ton'] = detailed_results['cost']/detailed_results['volume']
detailed_results.sort_values(by=['cost_per_ton'])

,facility_name,facility_location,pool_id,pool_location,distance,cost,n_trucks,industry,timeframe,volume,cost_per_ton
612,SAN JOAQUIN REFINING CO INC,"[35.39511, -119.04652]",Kern River::Kern River,"[35.447206, -118.98935600000002]",4.828655,1.105278e+05,5528.66180,oil_refining,30,138216.54500,7.996716e-01
792,Anaheim Combustion Turbine,"[33.8539, -117.8561]",Brea-Olinda::npool,"[33.930279, -117.86027]",5.282703,5.512416e+04,1995.37490,total,10,49884.37250,1.105039e+00
943,AES Redondo Beach,"[33.8504, -118.395]",Torrance:Onshore:Others,"[33.8237724, -118.3352684]",3.890536,1.179896e+04,370.31015,total,10,9257.75375,1.274495e+00
1095,"California Resources Elk Hills, LLC - Gas Proc...","[35.23893, -119.35951]",Elk Hills::Stevens (29R),"[35.295080525, -119.533703]",10.564826,1.860740e+04,433.57946,total,10,10839.48650,1.716631e+00
22,MARTINEZ REFINING COMPANY LLC,"[38.016594, -122.115392]",Ryer Island Gas:Onshore:pool,"[38.07513523, -122.01195049999998]",6.931118,4.862921e+05,10554.03500,oil_refining,20,263850.87500,1.843057e+00
...,...,...,...,...,...,...,...,...,...,...,...
790,"Ormond Beach Power, LLC","[34.1292, -119.1689]",Belridge North:pool,"[35.517082, -119.762591]",101.633552,3.255482e+09,106.90605,energy,20,2672.65125,1.218072e+06
420,LUNDAY-THAGARD COMPANY,"[33.946295, -118.16704]",Belridge North:pool,"[35.517082, -119.762591]",141.371663,3.255499e+09,106.90605,oil_refining,30,2672.65125,1.218079e+06
181,Phillips 66 Los Angeles Refinery - Wilmington ...,"[33.774469, -118.290696]",Belridge North:pool,"[35.517082, -119.762591]",146.612970,3.255501e+09,106.90605,total,20,2672.65125,1.218079e+06
113,Phillips 66 Los Angeles Refinery - Wilmington ...,"[33.774469, -118.290696]",Belridge North:pool,"[35.517082, -119.762591]",146.612970,3.255501e+09,106.90605,total,30,2672.65125,1.218079e+06


In [32]:
summary_results = detailed_results.groupby(by=['industry','timeframe']).agg(
    volume_transferred=('volume', 'sum'),
    total_cost=('cost', 'sum'),
    total_distance=('distance', 'sum')  
)
summary_results['cost_per_ton'] = summary_results['total_cost']/summary_results['volume_transferred']
summary_results



volume_transferred    total_cost  total_distance  \
industry     timeframe                                                     
cement       1                1.822768e+07  7.160608e+09     1913.577002   
             5                9.113839e+07  3.575466e+10     4070.699313   
             10               1.822768e+08  4.518601e+10     4603.963775   
             20               3.645536e+08  7.842704e+12    17039.395787   
             30               5.468304e+08  7.897318e+12    25906.140923   
energy       1                1.051721e+08  5.572413e+12    18160.637336   
             5                5.258606e+08  7.845148e+12    30825.783986   
             10               1.051721e+09  8.188888e+12    63431.238436   
             20               1.737409e+09  9.065173e+12   247290.896765   
             30               1.737409e+09  9.062199e+12   241248.161029   
oil_refining 1                6.605261e+07  5.483917e+12     3220.188503   
             5                3.302631e+08  7.805848e+12    13318.901286   
             10               6.605261e+08  7.999460e+12    34078.377399   
             20               1.321052e+09  8.331118e+12    51870.328789   
             30               1.737409e+09  9.054636e+12   180609.090932   
steel        1                1.430290e+04  7.566748e+06       45.538919   
             5                7.151450e+04  3.783374e+07       45.538919   
             10               1.430290e+05  7.566748e+07       45.538919   
             20               2.860580e+05  1.513350e+08       45.538919   
             30               4.290870e+05  2.270024e+08       45.538919   
total        1                1.894667e+08  7.735821e+12    27118.390417   
             5                9.473335e+08  8.139369e+12    67874.203997   
             10               1.737409e+09  9.062764e+12   257489.030551   
             20               1.737409e+09  9.061900e+12   266775.436916   
             30               1.737409e+09  9.054297e+12   283159.874391   

                        cost_per_ton  
industry     timeframe                
cement       1            392.842545  
             5            392.311689  
             10           247.897762  
             20         21513.173942  
             30         14441.988497  
energy       1          52983.752992  
             5          14918.683724  
             10          7786.178301  
             20          5217.640159  
             30          5215.928181  
oil_refining 1          83023.474591  
             5          23635.242031  
             10         12110.739163  
             20          6306.426028  
             30          5211.575510  
steel        1            529.035923  
             5            529.035923  
             10           529.035923  
             20           529.035923  
             30           529.035923  
total        1          40829.448257  
             5           8591.872294  
             10          5216.253342  
             20          5215.756214  
             30          5211.380171

In [33]:
detailed_results['volume'] = detailed_results['volume']/1000
detailed_results['cost'] = detailed_results['cost']/1000000

In [34]:
summary_results['volume_transferred'] = summary_results['volume_transferred']/1000
summary_results['total_cost'] = summary_results['total_cost']/1000000

In [35]:
detailed_results.sort_values(by=['cost_per_ton'])

,facility_name,facility_location,pool_id,pool_location,distance,cost,n_trucks,industry,timeframe,volume,cost_per_ton
612,SAN JOAQUIN REFINING CO INC,"[35.39511, -119.04652]",Kern River::Kern River,"[35.447206, -118.98935600000002]",4.828655,0.110528,5528.66180,oil_refining,30,138.216545,7.996716e-01
792,Anaheim Combustion Turbine,"[33.8539, -117.8561]",Brea-Olinda::npool,"[33.930279, -117.86027]",5.282703,0.055124,1995.37490,total,10,49.884372,1.105039e+00
943,AES Redondo Beach,"[33.8504, -118.395]",Torrance:Onshore:Others,"[33.8237724, -118.3352684]",3.890536,0.011799,370.31015,total,10,9.257754,1.274495e+00
1095,"California Resources Elk Hills, LLC - Gas Proc...","[35.23893, -119.35951]",Elk Hills::Stevens (29R),"[35.295080525, -119.533703]",10.564826,0.018607,433.57946,total,10,10.839486,1.716631e+00
22,MARTINEZ REFINING COMPANY LLC,"[38.016594, -122.115392]",Ryer Island Gas:Onshore:pool,"[38.07513523, -122.01195049999998]",6.931118,0.486292,10554.03500,oil_refining,20,263.850875,1.843057e+00
...,...,...,...,...,...,...,...,...,...,...,...
790,"Ormond Beach Power, LLC","[34.1292, -119.1689]",Belridge North:pool,"[35.517082, -119.762591]",101.633552,3255.482036,106.90605,energy,20,2.672651,1.218072e+06
420,LUNDAY-THAGARD COMPANY,"[33.946295, -118.16704]",Belridge North:pool,"[35.517082, -119.762591]",141.371663,3255.499029,106.90605,oil_refining,30,2.672651,1.218079e+06
181,Phillips 66 Los Angeles Refinery - Wilmington ...,"[33.774469, -118.290696]",Belridge North:pool,"[35.517082, -119.762591]",146.612970,3255.501270,106.90605,total,20,2.672651,1.218079e+06
113,Phillips 66 Los Angeles Refinery - Wilmington ...,"[33.774469, -118.290696]",Belridge North:pool,"[35.517082, -119.762591]",146.612970,3255.501270,106.90605,total,30,2.672651,1.218079e+06


In [36]:
list_results = detailed_results.values.tolist()
file_name = 'data/results/full_detailed_results.pkl'
open_file = open(file_name, "wb")
pickle.dump(list_results, open_file)
open_file.close()

html_results = detailed_results.to_html()
html_file = open("data/results/full_detailed_results.html", "w")
html_file.write(html_results)
html_file.close()

In [37]:
total_pool_volume = sum(pools_volumes)/1000

summary_results['percent_pool_volume_used'] = summary_results['volume_transferred'] / total_pool_volume
summary_results

volume_transferred    total_cost  total_distance  \
industry     timeframe                                                     
cement       1                1.822768e+04  7.160608e+03     1913.577002   
             5                9.113839e+04  3.575466e+04     4070.699313   
             10               1.822768e+05  4.518601e+04     4603.963775   
             20               3.645536e+05  7.842704e+06    17039.395787   
             30               5.468304e+05  7.897318e+06    25906.140923   
energy       1                1.051721e+05  5.572413e+06    18160.637336   
             5                5.258606e+05  7.845148e+06    30825.783986   
             10               1.051721e+06  8.188888e+06    63431.238436   
             20               1.737409e+06  9.065173e+06   247290.896765   
             30               1.737409e+06  9.062199e+06   241248.161029   
oil_refining 1                6.605261e+04  5.483917e+06     3220.188503   
             5                3.302631e+05  7.805848e+06    13318.901286   
             10               6.605261e+05  7.999460e+06    34078.377399   
             20               1.321052e+06  8.331118e+06    51870.328789   
             30               1.737409e+06  9.054636e+06   180609.090932   
steel        1                1.430290e+01  7.566748e+00       45.538919   
             5                7.151450e+01  3.783374e+01       45.538919   
             10               1.430290e+02  7.566748e+01       45.538919   
             20               2.860580e+02  1.513350e+02       45.538919   
             30               4.290870e+02  2.270024e+02       45.538919   
total        1                1.894667e+05  7.735821e+06    27118.390417   
             5                9.473335e+05  8.139369e+06    67874.203997   
             10               1.737409e+06  9.062764e+06   257489.030551   
             20               1.737409e+06  9.061900e+06   266775.436916   
             30               1.737409e+06  9.054297e+06   283159.874391   

                        cost_per_ton  percent_pool_volume_used  
industry     timeframe                                          
cement       1            392.842545                  0.010491  
             5            392.311689                  0.052457  
             10           247.897762                  0.104913  
             20         21513.173942                  0.209826  
             30         14441.988497                  0.314739  
energy       1          52983.752992                  0.060534  
             5          14918.683724                  0.302669  
             10          7786.178301                  0.605339  
             20          5217.640159                  1.000000  
             30          5215.928181                  1.000000  
oil_refining 1          83023.474591                  0.038018  
             5          23635.242031                  0.190089  
             10         12110.739163                  0.380179  
             20          6306.426028                  0.760358  
             30          5211.575510                  1.000000  
steel        1            529.035923                  0.000008  
             5            529.035923                  0.000041  
             10           529.035923                  0.000082  
             20           529.035923                  0.000165  
             30           529.035923                  0.000247  
total        1          40829.448257                  0.109051  
             5           8591.872294                  0.545257  
             10          5216.253342                  1.000000  
             20          5215.756214                  1.000000  
             30          5211.380171                  1.000000

In [38]:
list_summary_results = summary_results.values.tolist()
file_name = 'data/results/summary_results.pkl'
open_file = open(file_name, "wb")
pickle.dump(list_summary_results, open_file)
open_file.close()

html_summary_results = summary_results.to_html()
html_file = open("data/results/summary_results.html", "w")
html_file.write(html_summary_results)
html_file.close()